# AutoQuant

This notebook shows a working code example of how to use AIMET AutoQuant feature.

AIMET offers a suite of neural network post-training quantization techniques. Often, applying these techniques in a specific sequence, results in better accuracy and performance. Without the AutoQuant feature, the AIMET user needs to manually try out various combinations of AIMET quantization features. This manual process is error-prone and often time-consuming.

The AutoQuant feature, analyzes the model, determines the sequence of AIMET quantization techniques and applies these techniques. In addition, the user can specify the amount of accuracy drop that can be tolerated, in the AutoQuant API. As soon as this threshold accuracy is reached, AutoQuant stops applying any additional quantization technique. In summary, the AutoQuant feature saves time and automates the quantization of the neural networks.


#### Overall flow
This notebook covers the following
1. Instantiate the example evaluation and training pipeline
2. Load a pretrained FP32 model and apply the AutoQuant feature
3. Create a quantization simulation model (with fake quantization ops inserted) and evaluate this simuation model to get a quantized accuracy score

#### What this notebook is not
* This notebook is not designed to show state-of-the-art AutoQuant results. For example, it uses a relatively quantization-friendly model like Resnet18. Also, some optimization parameters like number of epochs to fine-tune are deliberately chosen to have the notebook execute more quickly.

---
## Dataset

This notebook relies on the ImageNet dataset for the task of image classification. If you already have a version of the dataset readily available, please use that. Else, please download the dataset from appropriate location (e.g. https://image-net.org/challenges/LSVRC/2012/index.php#).

**Note1**: The ImageNet dataset typically has the following characteristics and the dataloader provided in this example notebook rely on these
- Subfolders 'train' for the training samples and 'val' for the validation samples. Please see the [pytorch dataset description](https://pytorch.org/vision/0.8/_modules/torchvision/datasets/imagenet.html) for more details.
- A subdirectory per class, and a file per each image sample

**Note2**: To speed up the execution of this notebook, you may use a reduced subset of the ImageNet dataset. E.g. the entire ILSVRC2012 dataset has 1000 classes, 1000 training samples per class and 50 validation samples per class. But for the purpose of running this notebook, you could perhaps reduce the dataset to say 2 samples per class. This exercise is left upto the reader and is not necessary.

Edit the cell below and specify the directory where the downloaded ImageNet dataset is saved.

In [1]:
DATASET_DIR = '/prj/neo-nas/databatch/ILSVRC2012_PyTorch_tmp'         # Please replace this with a real directory

---
## 1. Example evaluation and training pipeline

The following is an example training and validation loop for this image classification task.

- **Does AIMET have any limitations on how the training, validation pipeline is written?** Not really. We will see later that AIMET will modify the user's model to create a QuantizationSim model which is still a PyTorch model. This QuantizationSim model can be used in place of the original model when doing inference or training.
- **Does AIMET put any limitation on the interface of the evaluate() or train() methods?** Not really. You should be able to use your existing evaluate and train routines as-is.


In [2]:
import os
import torch
from Examples.common import image_net_config
from Examples.torch.utils.image_net_evaluator import ImageNetEvaluator
from Examples.torch.utils.image_net_trainer import ImageNetTrainer
from Examples.torch.utils.image_net_data_loader import ImageNetDataLoader

class ImageNetDataPipeline:

    @staticmethod
    def get_val_dataloader() -> torch.utils.data.DataLoader:
        """
        Instantiates a validation dataloader for ImageNet dataset and returns it
        """
        data_loader = ImageNetDataLoader(DATASET_DIR,
                                         image_size=image_net_config.dataset['image_size'],
                                         batch_size=image_net_config.evaluation['batch_size'],
                                         is_training=False,
                                         num_workers=image_net_config.evaluation['num_workers']).data_loader
        return data_loader

    @staticmethod
    def evaluate(model: torch.nn.Module, use_cuda: bool) -> float:
        """
        Given a torch model, evaluates its Top-1 accuracy on the dataset
        :param model: the model to evaluate
        :param use_cuda: whether or not the GPU should be used.
        """
        evaluator = ImageNetEvaluator(DATASET_DIR, image_size=image_net_config.dataset['image_size'],
                                      batch_size=image_net_config.evaluation['batch_size'],
                                      num_workers=image_net_config.evaluation['num_workers'])

        return evaluator.evaluate(model, iterations=None, use_cuda=use_cuda)

    @staticmethod
    def finetune(model: torch.nn.Module, epochs, learning_rate, learning_rate_schedule, use_cuda):
        """
        Given a torch model, finetunes the model to improve its accuracy
        :param model: the model to finetune
        :param epochs: The number of epochs used during the finetuning step.
        :param learning_rate: The learning rate used during the finetuning step.
        :param learning_rate_schedule: The learning rate schedule used during the finetuning step.
        :param use_cuda: whether or not the GPU should be used.
        """
        trainer = ImageNetTrainer(DATASET_DIR, image_size=image_net_config.dataset['image_size'],
                                  batch_size=image_net_config.train['batch_size'],
                                  num_workers=image_net_config.train['num_workers'])

        trainer.train(model, max_epochs=epochs, learning_rate=learning_rate,
                      learning_rate_schedule=learning_rate_schedule, use_cuda=use_cuda)

In [3]:
import torchvision
from torchvision import datasets, transforms
from torchvision import transforms as T

def generate_dataloader(data, name,transform, batch_size=32):
    if data is None:
        return None

    if transform is None:
        dataset = torchvision.datasets.ImageFolder(data, transform=T.ToTensor())
    else:
        dataset = torchvision.datasets.ImageFolder(data, transform=transform)

    dataloader = DataLoader(dataset, batch_size=batch_size,shuffle=False)

    return dataloader

In [4]:
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
BATCH_SIZE = 100

transform = transforms.Compose((
    transforms.ToTensor(),
))

preprocess_transform_pretrain = T.Compose([
    T.Resize(256),  # Resize images to 256 x 256
    T.CenterCrop(224),  # Center crop image
    T.RandomHorizontalFlip(),
    T.ToTensor(),  # Converting cropped images to tensors
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])


img_net_dataloader = generate_dataloader(DATASET_DIR, "val",
                                 transform=preprocess_transform_pretrain, batch_size=BATCH_SIZE)

---
## 2. Load the model and evaluate to get a baseline FP32 accuracy score

For this example notebook, we are going to load a pretrained resnet18 model from torchvision. Similarly, you can load any pretrained PyTorch model instead.

In [5]:
from torchvision.models import resnet18

model = resnet18(pretrained=True)

AIMET quantization simulation requires the user's model definition to follow certain guidelines. For example, functionals defined in forward pass should be changed to equivalent torch.nn.Module.
AIMET user guide lists all these guidelines.
The following **ModelPreparer** API uses new graph transformation feature available in PyTorch 1.9+ version and automates model definition changes required to comply with the above guidelines. 

In [6]:
from aimet_torch.model_preparer import prepare_model

model = prepare_model(model)

2022-06-08 22:09:04,491 - root - INFO - AIMET
2022-06-08 22:09:04,528 - Quant - INFO - Functional         : Adding new module for node: {add} 
2022-06-08 22:09:04,529 - Quant - INFO - Reused/Duplicate   : Adding new module for node: {layer1_0_relu_1} 
2022-06-08 22:09:04,529 - Quant - INFO - Functional         : Adding new module for node: {add_1} 
2022-06-08 22:09:04,530 - Quant - INFO - Reused/Duplicate   : Adding new module for node: {layer1_1_relu_1} 
2022-06-08 22:09:04,531 - Quant - INFO - Functional         : Adding new module for node: {add_2} 
2022-06-08 22:09:04,531 - Quant - INFO - Reused/Duplicate   : Adding new module for node: {layer2_0_relu_1} 
2022-06-08 22:09:04,532 - Quant - INFO - Functional         : Adding new module for node: {add_3} 
2022-06-08 22:09:04,532 - Quant - INFO - Reused/Duplicate   : Adding new module for node: {layer2_1_relu_1} 
2022-06-08 22:09:04,533 - Quant - INFO - Functional         : Adding new module for node: {add_4} 
2022-06-08 22:09:04,533 -

/usr/local/lib/python3.6/dist-packages/torch/fx/graph.py:606: UserWarning: Attempted to insert a call_module Node with no underlying reference in the owning GraphModule! Call GraphModule.add_submodule to add the necessary submodule
  warnings.warn("Attempted to insert a call_module Node with "


---
We should decide whether to place the model on a CPU or CUDA device. This example code will use CUDA if available in your current execution environment. You can change this logic and force a device placement if needed.

In [7]:
use_cuda = False
if torch.cuda.is_available():
    print("YES. CUDA is available")
    use_cuda = True
    model.to(torch.device('cuda'))

YES. CUDA is available


---
Let's determine the FP32 (floating point 32-bit) accuracy of this model using the evaluate() routine

In [8]:
accuracy = ImageNetDataPipeline.evaluate(model, use_cuda)
print(accuracy)

2022-06-08 22:09:19,316 - Dataloader - INFO - Dataset consists of 2000 images in 1000 classes
2022-06-08 22:09:19,319 - Eval - INFO - No value of iteration is provided, running evaluation on complete dataset.
2022-06-08 22:09:19,320 - Eval - INFO - Evaluating nn.Module for 63 iterations with batch_size 32


100% (63 of 63) |########################| Elapsed Time: 0:00:11 Time:  0:00:11


2022-06-08 22:09:30,877 - Eval - INFO - Avg accuracy Top 1: 70.287698 Avg accuracy Top 5: 89.682540 on validation Dataset
70.28769841269842


---
## 3. Define Constants and Helper functions

In [9]:
import random

EVAL_DATASET_SIZE = 5000
CALIBRATION_DATASET_SIZE = 2000
BATCH_SIZE = 100

_subset_samplers = {}

def _create_sampled_data_loader(dataset, num_samples):
    if num_samples not in _subset_samplers:
        indices = random.sample(range(len(dataset)), num_samples)
        _subset_samplers[num_samples] = SubsetRandomSampler(indices=indices)
    return DataLoader(dataset,
                      sampler=_subset_samplers[num_samples],
                      batch_size=BATCH_SIZE)

## Prepare the Dataset

NOTE: In the actual use cases, a real dataset should provide by the users.

Now we use AIMET to create a QuantizationSimModel. This basically means that AIMET will insert fake quantization ops in the model graph and will configure them.
A few of the parameters are explained here
- **quant_scheme**: We set this to "training_range_learning_with_tf_init"
    - This is the key setting that enables "range learning". With this choice of quant scheme, AIMET will use the TF quant scheme to initialize the quantization parameters like scale/offset. And then those parameters are set to be trainable so they can continue to be updated during fine-tuning.
    - Another choice for quant_scheme is "training_range_learning_with_tf_enhanced_init". Similar to the above, but the initialization for scale/offset is doing using the TF Enhanced scheme. Since in both schemes the quantization parameters are set to be trainable, there is not much benefit to using this choice instead of "training_range_learning_with_tf_init.
- **default_output_bw**: Setting this to 8, essentially means that we are asking AIMET to perform all activation quantizations in the model using integer 8-bit precision
- **default_param_bw**: Setting this to 8, essentially means that we are asking AIMET to perform all parameter quantizations in the model using integer 8-bit precision

There are other parameters that are set to default values in this example. Please check the AIMET API documentation of QuantizationSimModel to see reference documentation for all the parameters.

In [10]:
from torchvision import datasets, transforms

input_shape = (1, 3, 224, 224)


transform = transforms.Compose((
    transforms.ToTensor(),
))

eval_dataset = datasets.FakeData(size=EVAL_DATASET_SIZE,
                                 image_size=input_shape[1:],
                                 num_classes=1000,
                                 transform=transform)



---
## Prepare unlabeled dataset
NOTE: In the actual use cases, the users should implement this part to serve
       their own goals if necessary.

In [11]:
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler

class UnlabeledDatasetWrapper(Dataset):
    def __init__(self, dataset):
        self._dataset = dataset

    def __len__(self):
        return len(self._dataset)

    def __getitem__(self, index):
        images, _ = self._dataset[index]
        return images

unlabeled_dataset = UnlabeledDatasetWrapper(eval_dataset)
unlabeled_data_loader = _create_sampled_data_loader(unlabeled_dataset, CALIBRATION_DATASET_SIZE)

img_net_data_set = UnlabeledDatasetWrapper(DATASET_DIR)
in_ds_dl = _create_sampled_data_loader(img_net_data_set, CALIBRATION_DATASET_SIZE)

---
Even though AIMET has added 'quantizer' nodes to the model graph but the model is not ready to be used yet. Before we can use the sim model for inference or training, we need to find appropriate scale/offset quantization parameters for each 'quantizer' node. For activation quantization nodes, we need to pass unlabeled data samples through the model to collect range statistics which will then let AIMET calculate appropriate scale/offset quantization parameters. This process is sometimes referred to as calibration. AIMET simply refers to it as 'computing encodings'.

So we create a routine to pass unlabeled data samples through the model. This should be fairly simple - use the existing train or validation data loader to extract some samples and pass them to the model. We don't need to compute any loss metric etc. So we can just ignore the model output for this purpose. A few pointers regarding the data samples
- In practice, we need a very small percentage of the overall data samples for computing encodings. For example, the training dataset for ImageNet has 1M samples. For computing encodings we only need 500 or 1000 samples.
- It may be beneficial if the samples used for computing encoding are well distributed. It's not necessary that all classes need to be covered etc. since we are only looking at the range of values at every layer activation. However, we definitely want to avoid an extreme scenario like all 'dark' or 'light' samples are used - e.g. only using pictures captured at night might not give ideal results.

The following shows an example of a routine that passes unlabeled samples through the model for computing encodings. This routine can be written in many different ways, this is just an example.

In [12]:
def pass_calibration_data(sim_model, use_cuda):
    data_loader = ImageNetDataPipeline.get_val_dataloader()
    batch_size = data_loader.batch_size

    if use_cuda:
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')

    sim_model.eval()
    samples = 1000

    batch_cntr = 0
    with torch.no_grad():
        for input_data, target_data in data_loader:

            inputs_batch = input_data.to(device)
            sim_model(inputs_batch)

            batch_cntr += 1
            if (batch_cntr * batch_size) > samples:
                break

---
Now we call AIMET to use the above routine to pass data through the model and then subsequently compute the quantization encodings. Encodings here refer to scale/offset quantization parameters.

## Prepare eval callback

In [13]:
from typing import Optional

def eval_callback(model: torch.nn.Module, num_samples: Optional[int] = None) -> float:
    if num_samples is None:
        num_samples = len(eval_dataset)

    eval_data_loader = _create_sampled_data_loader(eval_dataset, num_samples)

    num_correct_predictions = 0
    for images, labels in eval_data_loader:
        predictions = torch.argmax(model(images.cuda()), dim=1)
        num_correct_predictions += torch.sum(predictions.cpu() == labels)

    return int(num_correct_predictions) / num_samples

In [25]:
def eval_callback_2(model: torch.nn.Module, num_samples: Optional[int] = None) -> float:
    return ImageNetDataPipeline.evaluate(model, use_cuda)
    

---
## 4. Create AutoQuant  object

To perform quantization aware training (QAT), we simply train the model for a few more epochs (typically 15-20). As with any training job, hyper-parameters need to be searched for optimal results. Good starting points are to use a learning rate on the same order as the ending learning rate when training the original model, and to drop the learning rate by a factor of 10 every 5 epochs or so.

For the purpose of this example notebook, we are going to train only for 1 epoch. But feel free to change these parameters as you see fit.

In [26]:
from aimet_torch.auto_quant import AutoQuant

auto_quant = AutoQuant(allowed_accuracy_drop=0.01,
                       unlabeled_dataset_iterable=unlabeled_data_loader,
                       eval_callback=eval_callback_2)

## Optionally set AdaRound Parameters
The AutoQuant feature, internally uses default parameters to run the AdaRound step.
If and only if necessary, the daefault AdaRound Parameters could be modified using the API shown below.

In [27]:
from aimet_torch.adaround.adaround_weight import AdaroundParameters

ADAROUND_DATASET_SIZE = 2000
adaround_data_loader = _create_sampled_data_loader(unlabeled_dataset, ADAROUND_DATASET_SIZE)
adaround_params = AdaroundParameters(adaround_data_loader, num_batches=len(adaround_data_loader))
auto_quant.set_adaround_params(adaround_params)

---
Depending on your settings you may have observed a slight gain in accuracy after one epoch of training. Ofcourse, this was just an example. Please try this against the model of your choice and play with the hyper-parameters to get the best results.

So we have an improved model after QAT. Now the next step would be to actually take this model to target. For this purpose, we need to export the model with the updated weights without the fake quant ops. And also to export the encodings (scale/offset quantization parameters) that were updated during training since we employed QAT with range-learning. AIMET QuantizationSimModel provides an export API for this purpose.

## Run AutoQuant

In [28]:
dummy_input = torch.randn(input_shape)
model.eval()
model, accuracy, encoding_path =\
    auto_quant.apply(model.cuda(),
                     dummy_input_on_cpu=dummy_input.cpu(),
                     dummy_input_on_gpu=dummy_input.cuda())

2022-06-08 22:41:38,544 - AutoQuant - INFO - Starting AutoQuant
2022-06-08 22:41:40,610 - Dataloader - INFO - Dataset consists of 2000 images in 1000 classes
2022-06-08 22:41:40,612 - Eval - INFO - No value of iteration is provided, running evaluation on complete dataset.
2022-06-08 22:41:40,614 - Eval - INFO - Evaluating nn.Module for 63 iterations with batch_size 32


100% (63 of 63) |########################| Elapsed Time: 0:00:11 Time:  0:00:11


2022-06-08 22:41:52,117 - Eval - INFO - Avg accuracy Top 1: 70.287698 Avg accuracy Top 5: 89.682540 on validation Dataset
2022-06-08 22:41:52,119 - AutoQuant - INFO - Target eval score: 70.28
2022-06-08 22:41:52,119 - AutoQuant - INFO - FP32 eval score (W32A32): 70.29


2022-06-08 22:41:52,343 - Quant - INFO - No config file provided, defaulting to config file at /local/mnt/workspace/sundarr/os/aimet-mirror/build/staging/universal/lib/python/aimet_common/quantsim_config/default_config.json


2022-06-08 22:41:52,351 - Utils - INFO - ...... subset to store [Conv_0, Relu_1]


...

2022-06-08 23:09:37,811 - Quant - WARNING - Could not determine the last op in the sub-graph for layer=module_add, candidates=[]


2022-06-08 23:09:37,883 - AutoQuant - INFO - The results of AdaRound is saved in /tmp/.trace/adaround.pth and /tmp/.trace/adaround.encodings.


- AdaRound

2022-06-08 23:09:37,893 - AutoQuant - INFO - Session finished: AdaRound. (eval score: 6.94)


2022-06-08 23:09:38,018 - AutoQuant - INFO - Best eval score: 7.59
2022-06-08 23:09:38,019 - AutoQuant - INFO - AutoQuant is unable to match the target accuracy. Consider Quantization Aware Training.


In [29]:
print(accuracy)
print(encoding_path)

7.589285714285714
/tmp/.trace/batchnorm_folding.encodings


---
## Summary

Hope this notebook was useful for you to understand how to use AIMET for performing QAT with range-learning.

Few additional resources
- Refer to the AIMET API docs to know more details of the APIs and optional parameters
- Refer to the other example notebooks to understand how to use AIMET post-training quantization techniques and the vanilla QAT method (without range-learning)